## 前缀树
![trieNode](images/前缀树与贪心算法-前缀树.png)

In [1]:
class TrieNode:
  def __init__(self, value) -> None:
    self.value = value
    self.passes = 0
    self.ends = 0
    self.nexts = {}

class Trie:
  def __init__(self, array = None) -> None:
    self.root = TrieNode(None)
    if array:
      self.insert(array)
  def insert(self, array):
    for word in array:
      cur = self.root
      cur.passes += 1
      for char in word:
        if char in cur.nexts:
          cur = cur.nexts[char]
          cur.passes += 1
          continue
        cur.nexts[char] = TrieNode(char)
        cur = cur.nexts[char]
        cur.passes += 1
      cur.ends += 1
  def printStr(self, node, isEnd = False):
    if not node.nexts:
      return ' {' + '{}: End'.format(node.value) + '}'

    string = ' {' + '{}:'.format(node.value)
    length = len(node.nexts.values())
    index = 0
    for subNode in node.nexts.values():
      if index == length - 1:
        string += self.printStr(subNode, True)
      else:
        string += self.printStr(subNode, False)
      index += 1
    if not isEnd:
      string += '},'
    else:
      string += '}'
    return string

  def __str__(self) -> str:
    return self.printStr(self.root)

array = ['abc', 'ab', 'acb', 'bac']
theMap = Trie(array)
print(theMap)
print('passes of first \'a\':', theMap.root.nexts['a'].passes, '\nends of first \'a\':', theMap.root.nexts['a'].ends)
theMap.insert(['bca'])
print(theMap)

 {None: {a: {b: {c: End}}, {c: {b: End}}}, {b: {a: {c: End}}}},
passes of first 'a': 3 
ends of first 'a': 0
 {None: {a: {b: {c: End}}, {c: {b: End}}}, {b: {a: {c: End}}, {c: {a: End}}}},


## 贪心算法
在某一个标准下，优先考虑最满足标准的样本，最后考虑最不满足标准的样本，最终得到一个答案的算法，叫作贪心算法。  
也就是说，不从整体最优上加以考虑，所做出的是在某种意义上的局部最优解。  

### 贪心算法的在笔试时的解题套路
1. 实现--个不依靠贪心策略的解法X，可以用最暴力的尝试
2. 脑补出贪心策略A、贪心策略B、贪心策略C...
3. 用解法X和对数器，去验证每一个贪心策略，用实验的方式得知哪个贪心策略正确
4. 不要去纠结贪心策略的证明


### 例题1 
给一个字符串数组，将他们拼成一个字典序最小的大字符串。

In [2]:
from functools import cmp_to_key
def cmp_smallestOrder(x, y):
  return 1 if (x+y) > (y+x) else -1
  
def smallestOrder(array):
  array.sort(key = cmp_to_key(cmp_smallestOrder))

array = ['b', 'ba']
smallestOrder(array)
''.join(array)


'bab'

### 例题2
一块金条切成两半，是需要花费和长度数值一样的铜板的。  
比如长度为20的金条，不管切成长度多大的两半，都要花费20个铜板。  
一群人想整分整块金条，怎么分最省铜板?  
例如，给定数组{10, 20, 30}，代表一共三个人，整块金条长度为10+20+30=60.  
金条要分成10, 20, 30三个部分。如果先把长度60的金条分成10和50，花费60;  
再把长度50的金条分成20和30，花费50; 一共花费110铜板。  
但是如果先把长度60的金条分成30和30，花费60; 再把长度30金条分成10和20，花费30; 一共花费90铜板。  
输入一个数组，返回分割的最小代价。  

贪心策略：哈夫曼树

In [3]:
from queue import PriorityQueue

def cutGold(array):
  heap = PriorityQueue()
  for num in array:
    heap.put(num)
  
  sum = 0
  while heap.qsize() > 1:
    cur = heap.get() + heap.get()
    sum += cur
    heap.put(cur)

  return sum
array = [10,20,30]
cutGold(array)

90

### 例题3 皇后问题


In [5]:
class QueenQ:
  def __init__(self, n):
    if n < 1 or n > 32:
      return 0
    self.limit = -1 if n == 32 else (1 << n) - 1
    self.result = self.process(0,0,0)

  def process(self, column, leftLimit, rightLimit):
    # 如果所有列都过棋子，返回一次可能
    if column == self.limit:
      return 1

    # 目前可以摆放的位置，1为可摆放，0为不可摆放
    slots = self.limit & (~(column | leftLimit | rightLimit))
    result = 0
    while slots:
      # slot目前最右侧第一个1
      # e.g slot = 010010, mostRight = 000010
      mostRightOne = slots & (~slots + 1)

      # e.g slot - mostRight = 010010 - 000010 = 010000
      slots -= mostRightOne

      # 下次迭代
      result += self.process(
      column | mostRightOne, 
      (leftLimit | mostRightOne) << 1,
      (rightLimit | mostRightOne) >> 1,
      )
      
    return result

import time
start = time.time()
solve = QueenQ(10)
end = time.time()
# bin(solve.limit & (~solve.limit + 1))
print('Result: {}\nTime Cost: {}ms'.format(solve.result, (end - start)*1000))

Result: 724
Time Cost: 28.10811996459961ms
